<a href="https://colab.research.google.com/github/Futaba-Kosuke/STL10/blob/feature%2Ftrain/training/main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
 !ls

drive  images  sample_data  stl_net.pth


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image

# バージョン確認 (Google Colab default)
print(torch.__version__)  # 1.4.0
print(torchvision.__version__)  # 0.5.0
print(np.__version__)  # 1.18.2
print(matplotlib.__version__)  # 3.2.1
print(Image.__version__)  # 7.0.0

1.4.0
0.5.0
1.18.2
3.2.1
7.0.0


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [0]:
batch_size = 25
num_workers = 4

In [6]:
# データの読み込み
transform = {
  'train': transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
  ),
  'test': transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
  )
}
data_set = {
  x: torchvision.datasets.STL10(root='./images', split=x, download=True, transform=transform[x])
  for x in ('train', 'test')
}
data_size = {
  x: len(data_set[x]) for x in ('train', 'test')
}
print(data_size)

data_loaders = {
  x[0]: torch.utils.data.DataLoader(data_set[x[0]], batch_size=x[1], shuffle=x[2], num_workers=num_workers)
  for x in (('train', batch_size, True), ('test', 100, False))
}

Files already downloaded and verified
Files already downloaded and verified
{'train': 5000, 'test': 8000}


In [0]:
classes = ('airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck')

In [8]:
# モデル構築
from torchvision import models
from torch import nn

net = models.vgg16(pretrained=True)
last_in_features = net.classifier[6].in_features
net.classifier[6] = nn.Linear(last_in_features, 10)  # 最終層を10クラスに変更
net = net.to(device)
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [0]:
# 損失・最適関数の定義
from torch import optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)  # 10エポックごとに学習率が1/10に更新

In [0]:
# 学習用関数
def train_net(net, criterion, optimizer, scheduler, num_epochs):
  for epoch in range(num_epochs):
    print('Epoch %d/%d' % (epoch, num_epochs - 1))
    print('-' * 10)

    running_loss_sum = 0
    train_loss_sum = 0
    test_loss_sum = 0

    # モデルの更新
    net.train()
    for i, (inputs, labels) in enumerate(data_loaders['train']):
      inputs = inputs.to(device)
      labels = labels.to(device)
      # 勾配の初期化
      optimizer.zero_grad()
      # 予測
      outputs = net(inputs)
      # 損失の導出
      loss = criterion(outputs, labels)
      # 逆伝播
      loss.backward()
      # 勾配の更新
      optimizer.step()

      running_loss_sum += loss.item()
      train_loss_sum += loss.item()
      if i % 50 == 49:
        print('[%d] running_loss: %.3f' % (i + 1, running_loss_sum / 50))
        running_loss_sum = 0

    # モデルの評価
    net.eval()
    cnt_correct = 0
    for i, (inputs, labels) in enumerate(data_loaders['test']):
      inputs = inputs.to(device)
      labels = labels.to(device)

      # 予測
      outputs = net(inputs)
      # 損失の導出
      loss = criterion(outputs, labels)
      # lossの加算
      test_loss_sum += loss.item()

      _, predicted = torch.max(outputs, 1)
      is_correct = (predicted == labels).squeeze()
      
      for i in range(len(is_correct)):
        cnt_correct += is_correct[i].item()  # 正解なら1, 不正解なら0

    print('')
    print('train_loss_ave:\t%.3f' % (batch_size * train_loss_sum / data_size['train']))
    print('test_loss_ave:\t%.3f' % (100 * test_loss_sum / data_size['test']))
    print('test_accuracy: \t%.3f %%' % (100 * cnt_correct / data_size['test']))

    net.train()
    scheduler.step()

In [11]:
num_epochs=15
train_net(net, criterion, optimizer, scheduler, num_epochs)

Epoch 0/14
----------
[50] running_loss: 0.856
[100] running_loss: 0.473
[150] running_loss: 0.424
[200] running_loss: 0.344

train_loss_ave:	0.524
test_loss_ave:	0.357
test_accuracy: 	87.912 %
Epoch 1/14
----------
[50] running_loss: 0.221
[100] running_loss: 0.228
[150] running_loss: 0.228
[200] running_loss: 0.232

train_loss_ave:	0.227
test_loss_ave:	0.250
test_accuracy: 	91.862 %
Epoch 2/14
----------
[50] running_loss: 0.129
[100] running_loss: 0.127
[150] running_loss: 0.174
[200] running_loss: 0.115

train_loss_ave:	0.136
test_loss_ave:	0.236
test_accuracy: 	92.912 %
Epoch 3/14
----------
[50] running_loss: 0.114
[100] running_loss: 0.112
[150] running_loss: 0.096
[200] running_loss: 0.107

train_loss_ave:	0.107
test_loss_ave:	0.218
test_accuracy: 	92.888 %
Epoch 4/14
----------
[50] running_loss: 0.062
[100] running_loss: 0.054
[150] running_loss: 0.064
[200] running_loss: 0.065

train_loss_ave:	0.061
test_loss_ave:	0.275
test_accuracy: 	92.188 %
Epoch 5/14
----------
[50] run

In [0]:
# 保存
PATH = './drive/My Drive/stl_net.pth'
torch.save(net.state_dict(), PATH)

In [13]:
net.eval()

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for (inputs, labels) in data_loaders['test']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = net(inputs)
    
    _, predicted = torch.max(outputs, 1)
    is_correct = (predicted == labels).squeeze()

    for i in range(len(is_correct)):
      label = labels[i]
      class_correct[label] += is_correct[i].item()  # 正解なら1, 不正解なら0
      class_total[label] += 1

for i in range(10):
  print('Accuracy of %5s : %2d %%' % (
    classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of airplane : 96 %
Accuracy of  bird : 95 %
Accuracy of   car : 95 %
Accuracy of   cat : 88 %
Accuracy of  deer : 93 %
Accuracy of   dog : 89 %
Accuracy of horse : 94 %
Accuracy of monkey : 95 %
Accuracy of  ship : 98 %
Accuracy of truck : 94 %


In [14]:
net.eval()

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for (inputs, labels) in data_loaders['train']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = net(inputs)
    
    _, predicted = torch.max(outputs, 1)
    is_correct = (predicted == labels).squeeze()

    for i in range(len(is_correct)):
      label = labels[i]
      class_correct[label] += is_correct[i].item()  # 正解なら1, 不正解なら0
      class_total[label] += 1

for i in range(10):
  print('Accuracy of %5s : %2d %%' % (
    classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of airplane : 100 %
Accuracy of  bird : 100 %
Accuracy of   car : 100 %
Accuracy of   cat : 100 %
Accuracy of  deer : 100 %
Accuracy of   dog : 100 %
Accuracy of horse : 100 %
Accuracy of monkey : 100 %
Accuracy of  ship : 100 %
Accuracy of truck : 100 %
